In [1]:
import os
import math
import pandas as pd
import datetime as dt
import calendar
import numpy as np

In [2]:
dlst = [dt.datetime(2019,3,31,3,30), dt.datetime(2020,3,29,3,30), dt.datetime(2021,3,28,3,30), dt.datetime(2022,3,27,3,30)]
march_dlst_days = {2019: 31, 2020: 29, 2021: 28, 2022: 27, 2023: 26}

In [3]:
market_prices_dir = 'data/market/dam_idm_prices/'
unbalance_prices_dir = 'data/market/unbalance_prices/'


In [4]:
dam_idm_prices_files =  [file for file in os.listdir(market_prices_dir) if os.path.isfile(os.path.join(market_prices_dir, file))]
# dam_idm_prices_files.remove('desktop.ini')

print(len(dam_idm_prices_files))
print(dam_idm_prices_files)

ValueError: list.remove(x): x not in list

### Reading DAM prices

In [ ]:
file = 'price_DAM_IDM_01.2022.xls'
print(file)

sheet_name = 'Ціна_РДН_ОЕС_України'

date = file[-11:-4]
date = dt.datetime.strptime(date, '%m.%Y')

data = pd.read_excel(market_prices_dir + file, sheet_name=sheet_name, header=None)
data = data.drop(index=[0, 1, 2]).reset_index().drop(columns=['index', 0])
squized = data.stack()
print(squized)

squized = squized.reset_index(level=[0,1])
squized['level_0'] = squized['level_0'] + 1

squized.columns = ['day', 'hour','dam_price']    
squized = squized.astype({'day': 'int', 'hour': 'int', 'dam_price': 'float'})

if date.month == 3:
    #print('MONTH is MARCH!!!!')
    dlst_day = march_dlst_days[date.year]
    squized = squized.drop(squized[(squized.day == dlst_day) & (squized.hour == 24)].index)

start = dt.datetime(year=date.year, month=date.month, day=1, hour=0, minute=30)

last_month_day_calendar = calendar.monthrange(date.year, date.month)[-1]
last_month_day_data = squized['day'].max()

if last_month_day_data == last_month_day_calendar:
    year = date.year
    month = date.month + 1 
    if month == 13:
        month = 1
        year = year + 1
    end = dt.datetime(year=year, month=month, day=1, hour=0, minute=30) - dt.timedelta(hours=1)
else:
    end = dt.datetime(year=date.year, month=date.month, day=last_month_day_data + 1, hour=0, minute=30)
    end = end - dt.timedelta(hours=1)

#print(last_month_day_calendar, last_month_day_data)
#print(start, end)

index_in_kyiv = pd.date_range(start=start, end=end, freq='1H', tz='europe/kiev')
index_in_utc = index_in_kyiv.tz_convert('utc').tz_localize(None)

#print(squized.tail(52))

squized.index = index_in_utc
squized = squized.drop(columns=['day', 'hour'])

dam_prices = squized

#print(data.head(10))
print(date)
print('ok!')

print(dam_prices)


### Reading IDM prices

In [ ]:
file = file
print(file)

sheet_name = 'Ціна_ВДР_ОЕС_України'

idm_prices = dict()

date = file[-11:-4]
date = dt.datetime.strptime(date, '%m.%Y')

data = pd.read_excel(market_prices_dir + file, sheet_name=sheet_name, header=None)
data = data.drop(index=[0, 1, 2]).reset_index().drop(columns=['index', 0])

#data['datetime'] = dt.datetime(year=date.year, month=date.month, day=data[0])
squized = data.stack()

squized = squized.reset_index(level=[0,1])
squized['level_0'] = squized['level_0'] + 1

squized.columns = ['day', 'hour','idm_price']    
squized = squized.astype({'day': 'int', 'hour': 'int', 'idm_price': 'float'})

if date.month == 3:
    #print('MONTH is MARCH!!!!')
    dlst_day = march_dlst_days[date.year]
    squized = squized.drop(squized[(squized.day == dlst_day) & (squized.hour == 24)].index)

start = dt.datetime(year=date.year, month=date.month, day=1, hour=0, minute=30)

last_month_day_calendar = calendar.monthrange(date.year, date.month)[-1]
last_month_day_data = squized['day'].max()

if last_month_day_data == last_month_day_calendar:
    year = date.year
    month = date.month + 1 
    if month == 13:
        month = 1
        year = year + 1
    end = dt.datetime(year=year, month=month, day=1, hour=0, minute=30) - dt.timedelta(hours=1)
else:
    end = dt.datetime(year=date.year, month=date.month, day=last_month_day_data + 1, hour=0, minute=30)
    end = end - dt.timedelta(hours=1)

index_in_kyiv = pd.date_range(start=start, end=end, freq='1H', tz='europe/kiev')
index_in_utc = index_in_kyiv.tz_convert('utc').tz_localize(None)

#print(squized.tail(52))

squized.index = index_in_utc
squized = squized.drop(columns=['day', 'hour'])

idm_prices = squized
#print(data.head(10))
print(date)
print('ok!')

print(idm_prices)

In [ ]:
ub_prices_files =  [file for file in os.listdir(unbalance_prices_dir) if os.path.isfile(os.path.join(unbalance_prices_dir, file))]
ub_prices_files.remove('desktop.ini')

print(ub_prices_files)

### Reading IMSP prices

In [ ]:
file = 'Faktychni-tsiny-nebalansiv-01-18.01.2022.xlsx'
print(file)

sheet_name = 'Аркуш1'

ub_prices = dict()
    
date = file[-12:-5]
date = dt.datetime.strptime(date, '%m.%Y')

data = pd.read_excel(unbalance_prices_dir + file, index_col=[0, 1], usecols=[0, 1, 2], header=0, skiprows=[1, 2], parse_dates=[0])
data.columns = ['imsp']
data = data.astype({'imsp': 'float'})
time_deltas = [dt.timedelta(hours=int(record[-1][:2]), minutes=30) for record in data.index.values]
data.index = [record[0] + dt.timedelta(hours=int(record[-1][:2]), minutes=30) for record in data.index.values]
data.dropna(inplace=True)

if date.month == 3:
    print('MONTH is MARCH!!!!')
    dlst_day = march_dlst_days[date.year]
    print(dlst_day)
    #print(data)
    #print(data[(data.index.day == dlst_day) & (data.index.hour == 23)].index)
    #data = data.drop(data[(data.index.day == dlst_day) & (data.index.hour == 23)].index)
    #print(data)

#print(squized.tail(52))


start = dt.datetime(year=date.year, month=date.month, day=1, hour=0, minute=30)

last_month_day_calendar = calendar.monthrange(date.year, date.month)[-1]
last_month_day_data = data.index.day.max()

if last_month_day_data == last_month_day_calendar:
    year = date.year
    month = date.month + 1 
    if month == 13:
        month = 1
        year = year + 1
    end = dt.datetime(year=year, month=month, day=1, hour=0, minute=30) - dt.timedelta(hours=1)
else:
    end = dt.datetime(year=date.year, month=date.month, day=last_month_day_data, hour=23, minute=30)

#print(last_month_day_calendar, last_month_day_data)
#print(start, end)

index_in_kyiv = pd.date_range(start=start, end=end, freq='1H', tz='europe/kiev')
index_in_utc = index_in_kyiv.tz_convert('utc').tz_localize(None)

#print(squized.tail(52))

data.index = index_in_utc

ub_prices = data

print(date)
print('Ok!')

print(ub_prices)


### Preparing intersected data

In [ ]:
print(dam_prices.shape)
print(idm_prices.shape)
print(ub_prices.shape)

In [ ]:
print(dam_prices.loc[dam_prices.index.duplicated(keep='first')])
print(idm_prices.loc[idm_prices.index.duplicated(keep='first')])
print(ub_prices.loc[ub_prices.index.duplicated(keep='first')])

In [ ]:
prices = pd.concat([dam_prices, idm_prices, ub_prices], join='outer', axis=1)
prices = prices.dropna()
prices.index.name = 'datetime'

prices['positive_ub_price'] = (prices[['dam_price', 'imsp']].min(axis=1) * (1 - 0.05)).apply(lambda x: round(x, 2))
prices['negative_ub_price'] = (prices[['dam_price', 'imsp']].max(axis=1) * (1 + 0.05)).apply(lambda x: round(x, 2))
print(prices)
#prices['idm_price'] = idm_prices_1
#prices['imsp_price'] = ub_prices_1

#prices = dam_prices_1.merge(idm_prices_1, left_index=True, right_index=True).merge(ub_prices_1, left_index=True, right_index=True)
# prices.to_excel('prices.xlsx')
print('Ok!')
print(prices.info())
print(prices)

### Putting data into database

In [ ]:
from sqlalchemy import create_engine, MetaData, desc
from sqlalchemy.sql import select, insert, update, and_, or_, not_, func
from sqlalchemy.pool import NullPool
from settings.db import DO_URL


engine = create_engine(DO_URL, poolclass=NullPool)
metadata = MetaData()
metadata.reflect(bind=engine)

with engine.connect() as connection:

    emp_table = metadata.tables['electricity_market_prices']
    query_1 = select([emp_table.c.id, emp_table.c.completed]).where(and_(emp_table.c.year == date.year, emp_table.c.month == date.month))
    complete_period = prices.index.max().day == calendar.monthrange(date.year, date.month)[1]
    record_to_update = connection.execute(query_1).fetchall()
    print(record_to_update)
    if record_to_update:
        if not record_to_update[0][1]:
            update_statement = emp_table.update().values((record_to_update[0][0], date.year, date.month, 
                                                      list(prices.index), 
                                                      list(prices.dam_price), 
                                                      list(prices.idm_price), 
                                                      list(prices.imsp),
                                                      list(prices.positive_ub_price),
                                                      list(prices.negative_ub_price),
                                                      complete_period)).where(emp_table.c.id == record_to_update[0][0])
            updated_id = connection.execute(update_statement)
            print('Database record updated!')
        else:
            print('Not updated! Record already completed!')
    else:
        query_2 = select([func.max(emp_table.c.id)])
        max_id = connection.execute(query_2).fetchall()
        print(max_id)
        max_id = max_id[0][0] + 1 if max_id else 1
        print(max_id)

        insert_statement = emp_table.insert().values((max_id, date.year, date.month, 
                                                      list(prices.index), 
                                                      list(prices.dam_price), 
                                                      list(prices.idm_price), 
                                                      list(prices.imsp),
                                                      list(prices.positive_ub_price),
                                                      list(prices.negative_ub_price),
                                                      complete_period))
        inserted_id = connection.execute(insert_statement)
        print('Data inserted to database at index {}'.format(inserted_id))